In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display   #for loading and visualizing audio files
import IPython.display as ipd   #to play audio

plt.style.use('ggplot')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import time 
import pickle

### META-DATA inspection

In [3]:
genres = pd.read_csv('/Users/KarimChammaa/Desktop/GA/CAPSTONE_METADATA/fma_metadata/genres.csv')
raw_genres = pd.read_csv('/Users/KarimChammaa/Desktop/GA/CAPSTONE_METADATA/fma_metadata/raw_genres.csv')

features = pd.read_csv('/Users/KarimChammaa/Desktop/GA/CAPSTONE_METADATA/fma_metadata/features.csv')
#raw_features = pd.read_csv('/Users/KarimChammaa/Desktop/GA/CAPSTONE_METADATA/fma_metadata/raw_features.csv')

tracks = pd.read_csv('/Users/KarimChammaa/Desktop/GA/CAPSTONE_METADATA/fma_metadata/tracks.csv')
raw_tracks = pd.read_csv('/Users/KarimChammaa/Desktop/GA/CAPSTONE_METADATA/fma_metadata/raw_tracks.csv')

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,

In [7]:
genres

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
...,...,...,...,...,...
158,1032,60,102,Turkish,2
159,1060,30,46,Tango,2
160,1156,26,130,Fado,2
161,1193,72,763,Christmas,38


In [11]:
raw_genres

,genre_id,genre_color,genre_handle,genre_parent_id,genre_title
0,1,#006666,Avant-Garde,38.0,Avant-Garde
1,2,#CC3300,International,NaN,International
2,3,#000099,Blues,NaN,Blues
3,4,#990099,Jazz,NaN,Jazz
4,5,#8A8A65,Classical,NaN,Classical
...,...,...,...,...,...
159,1032,#CC3300,Turkish,102.0,Turkish
160,1060,#CC3300,tango,46.0,Tango
161,1156,#CC3300,Fado,130.0,Fado
162,1193,#D4A017,Christmas,763.0,Christmas


In [19]:
pd.set_option('display.max_columns', None)
features.head(40)

,feature,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,chroma_cens.9,chroma_cens.10,chroma_cens.11,chroma_cens.12,chroma_cens.13,chroma_cens.14,chroma_cens.15,chroma_cens.16,chroma_cens.17,chroma_cens.18,chroma_cens.19,chroma_cens.20,chroma_cens.21,chroma_cens.22,chroma_cens.23,chroma_cens.24,chroma_cens.25,chroma_cens.26,chroma_cens.27,chroma_cens.28,chroma_cens.29,chroma_cens.30,chroma_cens.31,chroma_cens.32,chroma_cens.33,chroma_cens.34,chroma_cens.35,chroma_cens.36,chroma_cens.37,chroma_cens.38,chroma_cens.39,chroma_cens.40,chroma_cens.41,chroma_cens.42,chroma_cens.43,chroma_cens.44,chroma_cens.45,chroma_cens.46,chroma_cens.47,chroma_cens.48,chroma_cens.49,chroma_cens.50,chroma_cens.51,chroma_cens.52,chroma_cens.53,chroma_cens.54,chroma_cens.55,chroma_cens.56,chroma_cens.57,chroma_cens.58,chroma_cens.59,chroma_cens.60,chroma_cens.61,chroma_cens.62,chroma_cens.63,chroma_cens.64,chroma_cens.65,chroma_cens.66,chroma_cens.67,chroma_cens.68,chroma_cens.69,chroma_cens.70,chroma_cens.71,chroma_cens.72,chroma_cens.73,chroma_cens.74,chroma_cens.75,chroma_cens.76,chroma_cens.77,chroma_cens.78,chroma_cens.79,chroma_cens.80,chroma_cens.81,chroma_cens.82,chroma_cens.83,chroma_cqt,chroma_cqt.1,chroma_cqt.2,chroma_cqt.3,chroma_cqt.4,chroma_cqt.5,chroma_cqt.6,chroma_cqt.7,chroma_cqt.8,chroma_cqt.9,chroma_cqt.10,chroma_cqt.11,chroma_cqt.12,chroma_cqt.13,chroma_cqt.14,chroma_cqt.15,chroma_cqt.16,chroma_cqt.17,chroma_cqt.18,chroma_cqt.19,chroma_cqt.20,chroma_cqt.21,chroma_cqt.22,chroma_cqt.23,chroma_cqt.24,chroma_cqt.25,chroma_cqt.26,chroma_cqt.27,chroma_cqt.28,chroma_cqt.29,chroma_cqt.30,chroma_cqt.31,chroma_cqt.32,chroma_cqt.33,chroma_cqt.34,chroma_cqt.35,chroma_cqt.36,chroma_cqt.37,chroma_cqt.38,chroma_cqt.39,chroma_cqt.40,chroma_cqt.41,chroma_cqt.42,chroma_cqt.43,chroma_cqt.44,chroma_cqt.45,chroma_cqt.46,chroma_cqt.47,chroma_cqt.48,chroma_cqt.49,chroma_cqt.50,chroma_cqt.51,chroma_cqt.52,chroma_cqt.53,chroma_cqt.54,chroma_cqt.55,chroma_cqt.56,chroma_cqt.57,chroma_cqt.58,chroma_cqt.59,chroma_cqt.60,chroma_cqt.61,chroma_cqt.62,chroma_cqt.63,chroma_cqt.64,chroma_cqt.65,chroma_cqt.66,chroma_cqt.67,chroma_cqt.68,chroma_cqt.69,chroma_cqt.70,chroma_cqt.71,chroma_cqt.72,chroma_cqt.73,chroma_cqt.74,chroma_cqt.75,chroma_cqt.76,chroma_cqt.77,chroma_cqt.78,chroma_cqt.79,chroma_cqt.80,chroma_cqt.81,chroma_cqt.82,chroma_cqt.83,chroma_stft,chroma_stft.1,chroma_stft.2,chroma_stft.3,chroma_stft.4,chroma_stft.5,chroma_stft.6,chroma_stft.7,chroma_stft.8,chroma_stft.9,chroma_stft.10,chroma_stft.11,chroma_stft.12,chroma_stft.13,chroma_stft.14,chroma_stft.15,chroma_stft.16,chroma_stft.17,chroma_stft.18,chroma_stft.19,chroma_stft.20,chroma_stft.21,chroma_stft.22,chroma_stft.23,chroma_stft.24,chroma_stft.25,chroma_stft.26,chroma_stft.27,chroma_stft.28,chroma_stft.29,chroma_stft.30,chroma_stft.31,chroma_stft.32,chroma_stft.33,chroma_stft.34,chroma_stft.35,chroma_stft.36,chroma_stft.37,chroma_stft.38,chroma_stft.39,chroma_stft.40,chroma_stft.41,chroma_stft.42,chroma_stft.43,chroma_stft.44,chroma_stft.45,chroma_stft.46,chroma_stft.47,chroma_stft.48,chroma_stft.49,chroma_stft.50,chroma_stft.51,chroma_stft.52,chroma_stft.53,chroma_stft.54,chroma_stft.55,chroma_stft.56,chroma_stft.57,chroma_stft.58,chroma_stft.59,chroma_stft.60,chroma_stft.61,chroma_stft.62,chroma_stft.63,chroma_stft.64,chroma_stft.65,chroma_stft.66,chroma_stft.67,chroma_stft.68,chroma_stft.69,chroma_stft.70,chroma_stft.71,chroma_stft.72,chroma_stft.73,chroma_stft.74,chroma_stft.75,chroma_stft.76,chroma_stft.77,chroma_stft.78,chroma_stft.79,chroma_stft.80,chroma_stft.81,chroma_stft.82,chroma_stft.83,mfcc,mfcc.1,mfcc.2,mfcc.3,mfcc.4,mfcc.5,mfcc.6,mfcc.7,mfcc.8,mfcc.9,mfcc.10,mfcc.11,mfcc.12,mfcc.13,mfcc.14,mfcc.15,mfcc.16,mfcc.17,mfcc.18,mfcc.19,mfcc.20,mfcc.21,mfcc.22,mfcc.23,mfcc.24,mfcc.25,mfcc.26,mfcc.27,mfcc.28,mfcc.29,mfcc.30,mfcc.31,mfcc.32,mfcc.33,mfcc.34,mfcc.35,mfcc.36,mfcc.37,mfcc.38,mfcc.39,mfcc.40,mfcc.41,mfcc.42

In [14]:
features.head()

,track_id,album_id,album_title,album_url,artist_id,artist_name,artist_url,artist_website,license_image_file,license_image_file_large,...,track_information,track_instrumental,track_interest,track_language_code,track_listens,track_lyricist,track_number,track_publisher,track_title,track_url
0,2,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,4656,en,1293,NaN,3,NaN,Food,http://freemusicarchive.org/music/AWOL/AWOL_-_...
1,3,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,1470,en,514,NaN,4,NaN,Electric Ave,http://freemusicarchive.org/music/AWOL/AWOL_-_...
2,5,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,1933,en,1151,NaN,6,NaN,This World,http://freemusicarchive.org/music/AWOL/AWOL_-_...
3,10,6.0,Constant Hitmaker,http://freemusicarchive.org/music/Kurt_Vile/Co...,6,Kurt Vile,http://freemusicarchive.org/music/Kurt_Vile/,http://kurtvile.com,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,54881,en,50135,NaN,1,NaN,Freeway,http://freemusicarchive.org/music/Kurt_Vile/Co...
4,20,4.0,Niris,http://freemusicarchive.org/music/Chris_and_Ni...,4,Nicky Cook,http://freemusicarchive.org/music/Chris_and_Ni...,NaN,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,978,en,361,NaN,3,NaN,Spiritual Level,http://freemusicarchive.org/music/Chris_and_Ni...


In [26]:
tracks.iloc[10050:10100]

,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,album.9,album.10,album.11,album.12,artist,artist.1,artist.2,artist.3,artist.4,artist.5,artist.6,artist.7,artist.8,artist.9,artist.10,artist.11,artist.12,artist.13,artist.14,artist.15,artist.16,set,set.1,track,track.1,track.2,track.3,track.4,track.5,track.6,track.7,track.8,track.9,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
10050,16600,0,2009-07-14 11:52:01,2005-08-22 00:00:00,NaN,0,3802,<p>Nambavan lives in the Russian republic of T...,10575,NaN,[],Chechnya on a dance floor,4,Album,NaN,NaN,NaN,<p>Nambavan unites various musical styles: Ita...,0,2009-05-25 04:05:05,12,3761,NaN,NaN,NaN,NaN,Nambavan,NaN,['nambavan'],http://www.wmrecordings.com/freedownloads.htm,NaN,validation,large,160000,0,NaN,2009-07-14 11:51:55,NaN,147,2,NaN,"[11, 15, 181, 182, 296]","[296, 11, 14, 15, 181, 182]",NaN,5402,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1961,NaN,4,NaN,[],Vtoraya Lubov
10051,16601,0,2009-07-14 11:57:11,2005-08-14 00:00:00,NaN,2,3803,"<p>Ever since René Vis [1959] heard music, dee...",3281,NaN,[],Captured in realica,9,Album,NaN,NaN,NaN,NaN,1,2009-06-29 02:01:35,3,4349,52.152355,The Netherlands,5.29297,NaN,René Vis,NaN,['ren vis'],http://www.wmrecordings.com/releases/wm052.htm,NaN,training,large,192000,0,NaN,2009-07-14 11:57:07,NaN,296,2,NaN,"[10, 15, 76, 362]","[10, 76, 362, 15]",NaN,5534,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,484,NaN,1,NaN,[],Over The Mountains
10052,16602,0,2009-07-14 11:57:11,2005-08-14 00:00:00,NaN,2,3803,"<p>Ever since René Vis [1959] heard music, dee...",3281,NaN,[],Captured in realica,9,Album,NaN,NaN,NaN,NaN,1,2009-06-29 02:01:35,3,4349,52.152355,The Netherlands,5.29297,NaN,René Vis,NaN,['ren vis'],http://www.wmrecordings.com/releases/wm052.htm,NaN,training,large,192000,0,NaN,2009-07-14 11:57:08,NaN,153,1,NaN,"[10, 15, 76, 362]","[10, 76, 362, 15]",NaN,5440,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,368,NaN,2,NaN,[],Mister cool
10053,16603,0,2009-07-14 11:57:11,2005-08-14 00:00:00,NaN,2,3803,"<p>Ever since René Vis [1959] heard music, dee...",3281,NaN,[],Captured in realica,9,Album,NaN,NaN,NaN,NaN,1,2009-06-29 02:01:35,3,4349,52.152355,The Netherlands,5.29297,NaN,René Vis,NaN,['ren vis'],http://www.wmrecordings.com/releases/wm052.htm,NaN,training,large,-1,0,NaN,2009-07-14 07:57:07,NaN,144,1,NaN,"[10, 15, 76, 362]","[10, 76, 362, 15]",NaN,4755,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,376,NaN,3,NaN,[],Fridaynight Forever
10054,16604,0,2009-07-14 11:57:11,2005-08-14 00:00:00,NaN,2,3803,"<p>Ever since René Vis [1959] heard music, dee...",3281,NaN,[],Captured in realica,9,Album,NaN,NaN,NaN,NaN,1,2009-06-29 02:01:35,3,4349,52.152355,The Netherlands,5.29297,NaN,René Vis,NaN,['ren vis'],http://www.wmrecordings.com/releases/wm052.htm,NaN,training,large,192000,0,NaN,2009-07-14 11:57:09,NaN,136,1,NaN,"[10, 15, 76, 362]","[10, 76, 362, 15]",NaN,4230,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,265,NaN,4,NaN,[],Back To Radio
10055,16605,0,2009-07-14 11:57:11,2005-08-14 00:00:00,NaN,2,3803,"<p>Ever since René Vis [1959] heard music, dee...",3281,NaN,[],Captured in realica,9,Album,NaN,NaN,NaN,NaN,1,2009-06-29 02:01:35,3,4349,52.152355,The Netherlands,5.29297,NaN,René Vis,NaN,['ren vis'],http://www.wmrecordings.com/releases/wm052.htm,NaN,training,large,192000,0,NaN,2009-07-14 11:57:10,NaN,152,1,NaN,"[10, 15, 76, 362]","[10, 76, 362, 15]",NaN,3680,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,151,NaN,5,NaN,[],Captured In Realica
10056,16606,0,2009-07-14 11:57:11,2005-08-14 00:00:00,NaN,2,3803,"<p>Ever since René Vis [1959] heard music, dee...",3281,NaN,[],Captured in realica,9,Album,NaN,NaN,NaN,NaN,1,2009-06-29 02:01:35,3,4349,52.152355,The Netherlands,5.29297,NaN,René Vis,NaN,['ren vis'],http://www.wmrecordings.com/releases/wm052.htm,NaN,training,large,192000,0,NaN,2009-07-14 11:57:10,NaN,240,2,NaN,"[10, 15, 76, 362]","[10, 76, 362, 15]",NaN

In [9]:
pd.set_option('display.max_columns', None)
tracks.head(50)

,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,album.9,album.10,album.11,album.12,artist,artist.1,artist.2,artist.3,artist.4,artist.5,artist.6,artist.7,artist.8,artist.9,artist.10,artist.11,artist.12,artist.13,artist.14,artist.15,artist.16,set,set.1,track,track.1,track.2,track.3,track.4,track.5,track.6,track.7,track.8,track.9,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
0,NaN,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,comments,date_created,favorites,id,latitude,location,longitude,members,name,related_projects,tags,website,wikipedia_page,split,subset,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
1,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.0583238,New Jersey,-74.4056612,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,0,NaN,2008-11-26 01:48:12,2008-11-26 00:00:00,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.0583238,New Jersey,-74.4056612,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,medium,256000,0,NaN,2008-11-26 01:48:14,2008-11-26 00:00:00,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
4,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.0583238,New Jersey,-74.4056612,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,0,NaN,2008-11-26 01:48:20,2008-11-26 00:00:00,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
5,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,NaN,[],Constant Hitmaker,2,Album,NaN,NaN,"Mexican Summer, Richie Records, Woodsist, Skul...","<p><span style=""font-family:Verdana, Geneva, A...",3,2008-11-26 01:42:55,74,6,NaN,NaN,NaN,"Kurt Vile, the Violators",Kurt Vile,NaN,"['philly', 'kurt vile']",http://kurtvile.com,NaN,training,small,192000,0,Kurt Vile,2008-11-25 17:49:06,2008-11-26 00:00:00,161,178,Pop,[10],[10],NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
6,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,[],Niris,13,Album,1990-01-01 00:00:00,2011-01-01 00:00:00,NaN,<p>Songs written by: Nicky Cook</p>\n<p>VOCALS...,2,2008-11-26 01:42:52,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,"['instrumentals', 'experimental pop', 'post pu...",NaN,NaN,training,large,256000,0,NaN,2008-11-26 01:48:56,2008-01-01 00:00:00,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,Na

In [7]:
pd.set_option('display.max_columns', None)
tracks[tracks['Unnamed: 0'] == '593']

,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,album.9,album.10,album.11,album.12,artist,artist.1,artist.2,artist.3,artist.4,artist.5,artist.6,artist.7,artist.8,artist.9,artist.10,artist.11,artist.12,artist.13,artist.14,artist.15,artist.16,set,set.1,track,track.1,track.2,track.3,track.4,track.5,track.6,track.7,track.8,track.9,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
370,593,2,2008-11-26 02:10:00,2009-01-08 00:00:00,NaN,3,157,"<p>Another Bliss Blood collab, this time featu...",11035,NaN,[],Finally Here,4,Album,NaN,NaN,NaN,<p>Blues band formed by Bliss Blood (of the Mo...,0,2008-11-26 02:07:36,5,113,40.65,"Brooklyn, NY",-73.95,"Bliss Blood: vocals, tenor guitar\nMarty Barto...",Delta Dreambox,NaN,['delta dreambox'],blissblood.com,NaN,training,medium,160000,0,NaN,2008-11-26 02:06:21,2006-01-01 00:00:00,189,10,Jazz,[37],"[4, 37]",NaN,4806,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,3092,NaN,4,NaN,[],Finally Here


In [29]:
pd.set_option('display.max_columns', None)
tracks.head(30)

,Unnamed: 0,album,album.1,album.2,album.3,album.4,album.5,album.6,album.7,album.8,album.9,album.10,album.11,album.12,artist,artist.1,artist.2,artist.3,artist.4,artist.5,artist.6,artist.7,artist.8,artist.9,artist.10,artist.11,artist.12,artist.13,artist.14,artist.15,artist.16,set,set.1,track,track.1,track.2,track.3,track.4,track.5,track.6,track.7,track.8,track.9,track.10,track.11,track.12,track.13,track.14,track.15,track.16,track.17,track.18,track.19
0,NaN,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,comments,date_created,favorites,id,latitude,location,longitude,members,name,related_projects,tags,website,wikipedia_page,split,subset,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
1,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.0583238,New Jersey,-74.4056612,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,0,NaN,2008-11-26 01:48:12,2008-11-26 00:00:00,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.0583238,New Jersey,-74.4056612,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,medium,256000,0,NaN,2008-11-26 01:48:14,2008-11-26 00:00:00,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
4,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.0583238,New Jersey,-74.4056612,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000,0,NaN,2008-11-26 01:48:20,2008-11-26 00:00:00,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
5,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,NaN,[],Constant Hitmaker,2,Album,NaN,NaN,"Mexican Summer, Richie Records, Woodsist, Skul...","<p><span style=""font-family:Verdana, Geneva, A...",3,2008-11-26 01:42:55,74,6,NaN,NaN,NaN,"Kurt Vile, the Violators",Kurt Vile,NaN,"['philly', 'kurt vile']",http://kurtvile.com,NaN,training,small,192000,0,Kurt Vile,2008-11-25 17:49:06,2008-11-26 00:00:00,161,178,Pop,[10],[10],NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
6,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,[],Niris,13,Album,1990-01-01 00:00:00,2011-01-01 00:00:00,NaN,<p>Songs written by: Nicky Cook</p>\n<p>VOCALS...,2,2008-11-26 01:42:52,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,"['instrumentals', 'experimental pop', 'post pu...",NaN,NaN,training,large,256000,0,NaN,2008-11-26 01:48:56,2008-01-01 00:00:00,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,Na

In [19]:
raw_tracks

,track_id,album_id,album_title,album_url,artist_id,artist_name,artist_url,artist_website,license_image_file,license_image_file_large,...,track_information,track_instrumental,track_interest,track_language_code,track_listens,track_lyricist,track_number,track_publisher,track_title,track_url
0,2,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,4656,en,1293,NaN,3,NaN,Food,http://freemusicarchive.org/music/AWOL/AWOL_-_...
1,3,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,1470,en,514,NaN,4,NaN,Electric Ave,http://freemusicarchive.org/music/AWOL/AWOL_-_...
2,5,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,1933,en,1151,NaN,6,NaN,This World,http://freemusicarchive.org/music/AWOL/AWOL_-_...
3,10,6.0,Constant Hitmaker,http://freemusicarchive.org/music/Kurt_Vile/Co...,6,Kurt Vile,http://freemusicarchive.org/music/Kurt_Vile/,http://kurtvile.com,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,54881,en,50135,NaN,1,NaN,Freeway,http://freemusicarchive.org/music/Kurt_Vile/Co...
4,20,4.0,Niris,http://freemusicarchive.org/music/Chris_and_Ni...,4,Nicky Cook,http://freemusicarchive.org/music/Chris_and_Ni...,NaN,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,978,en,361,NaN,3,NaN,Spiritual Level,http://freemusicarchive.org/music/Chris_and_Ni...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109722,155316,22940.0,"Live at Monty Hall, 2/17/2017",http://freemusicarchive.org/music/Spowder/Live...,24357,Spowder,http://freemusicarchive.org/music/Spowder/,https://spowder.bandcamp.com/,http://i.creativecommons.org/l/by-nc-nd/4.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,122,NaN,102,NaN,3,NaN,The Auger,http://freemusicarchive.org/music/Spowder/Live...
109723,155317,22940.0,"Live at Monty Hall, 2/17/2017",http://freemusicarchive.org/music/Spowder/Live...,24357,Spowder,http://freemusicarchive.org/music/Spowder/,https://spowder.bandcamp.com/,http://i.creativecommons.org/l/by-nc-nd/4.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,194,NaN,165,NaN,4,NaN,Let's Skin Ruby,http://freemusicarchive.org/music/Spowder/Live...
109724,155318,22940.0,"Live at Monty Hall, 2/17/2017",http://freemusicarchive.org/music/Spowder/Live...,24357,Spowder,http://freemusicarchive.org/music/Spowder/,https://spowder.bandcamp.com/,http://i.creativecommons.org/l/by-nc-nd/4.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,214,NaN,168,NaN,6,NaN,My House Smells Like Kim Deal/Pulp,http://freemusicarchive.org/music/Spowder/Live...
109725,155319,22940.0,"Live at Monty Hall, 2/17/2017",http://freemusicarchive.org/music/Spowder/Live...,24357,Spowder,http://freemusicarchive.org/music/Spowder/,https://spowder.bandcamp.com/,http://i.creativecommons.org/l/by-nc-nd/4.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,336,NaN,294,NaN,5,NaN,The Man With Two Mouths,http://freemusicarchive.org/music/Spowder/Live...


# Notes

Start writing up presentation as you go after modelling stage

## 3 methods:

**A.** PCA each feature (spectrogram) ---> save each PCA'd feature into own dataframe ---> model on each one individually --> majority vote predictions

**B.** From PCA'd dataframe combine into one big and model entirely ---> predictions

**C.** Do descriptive stats and model on this ---> predictions


- simple model on various features count predictions and choose from majority vote

(chroma, tempo, spectogram, mse)
- MAKE SURE each AUDIO file is same sample rate!


- turn into black & white images, pca

**N. of frames from hop length and y**

If you divide your y_shape length by the hop_length - you get the number of the frames. 

- y.shape/hop_length

**Regarding warning message :**

There is nothing to fix: this is operating as intended.

Again, this is not an error, it is a warning. Your code will work just fine, it just will fall back on the (slower) audioread decoder. This is unavoidable for now, since libsndfile still does not support mp3.

If you're really concerned about speed, you could transcode your mp3s into a more permissive format (eg ogg vorbis

In [ ]:
'''
chroma_stft([y, sr, S, norm, n_fft, …])            Compute a chromagram from a waveform or power spectrogram.

chroma_cqt([y, sr, C, hop_length, fmin, …])        Constant-Q chromagram

chroma_cens([y, sr, C, hop_length, fmin, …])       Computes the chroma variant “Chroma Energy Normalized” (CENS)

melspectrogram([y, sr, S, n_fft, …])               Compute a mel-scaled spectrogram.

mfcc([y, sr, S, n_mfcc, dct_type, norm, lifter])   Mel-frequency cepstral coefficients (MFCCs)

rms([y, S, frame_length, hop_length, …])           Compute root-mean-square (RMS) value for each frame, either from the audio samples y or from a spectrogram S.

spectral_centroid([y, sr, S, n_fft, …])            Compute the spectral centroid.

spectral_bandwidth([y, sr, S, n_fft, …])           Compute p’th-order spectral bandwidth.

spectral_contrast([y, sr, S, n_fft, …])            Compute spectral contrast

spectral_flatness([y, S, n_fft, hop_length, …])    Compute spectral flatness

spectral_rolloff([y, sr, S, n_fft, …])             Compute roll-off frequency.

poly_features([y, sr, S, n_fft, hop_length, …])    Get coefficients of fitting an nth-order polynomial to the columns of a spectrogram.

tonnetz([y, sr, chroma])                           Computes the tonal centroid features (tonnetz)

zero_crossing_rate(y[, frame_length, …])           Compute the zero-crossing rate of an audio time series.
'''

Old way to collect audio time series and create dataframe 

As it is lower to retrieve from pandas dataframe, decided to use pickle dump to store numpy array files on harddrive

In [ ]:
#get all filepaths for a given folder and store the audio time series y, sr = librosa.load(test_filepath, duration =25)

fp_main = '/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/'
folders = os.listdir(fp_main)

numbers = ['0','1','2','3','4','5','6','7','8','9']

track_id_list = []
audio_timeseries_list = []  #saved as a tuple (y,sr)
filepath_list = []
folder_list = []
error_list = []

#tranch this with finns code
for folder_numb in folders:
    if folder_numb == '101': # limit till what file number you want to collect. 
        break
    if any(x in folder_numb for x in numbers):  #to prevent picking up hidden files such as 'README.txt' or 'checksums'
        print("folder :",folder_numb)
        fp_main_new = fp_main + folder_numb + '/'  #not forget the '/' at the end
        audio_clips = os.listdir(fp_main_new)
        for track_id in audio_clips:
            if '_' in track_id:
                pass
            else:
                fp_new = fp_main_new + track_id    
                try:                #need try and except as y,sr extraction doesnt work for all e.g '001486.mp3'
                    #fp_new = fp_main_new + track_id              #fp for filepath
                    y,sr = librosa.load(fp_new, duration = 30) #clips are 30secs
    
                    track_id_list.append(track_id.split('.')[0])
                    audio_timeseries_list.append((y,sr)) 
                    error_list.append('none') 
                    filepath_list.append(fp_new) 
                    folder_list.append(folder_numb)
                except:
                    track_id_list.append(track_id.split('.')[0]) #track_id
                    audio_timeseries_list.append('error') #numpy array
                    error_list.append('error')
                    filepath_list.append(fp_new) 
                    folder_list.append(folder_numb)
    else:
        pass 

In [ ]:
df_time_series = pd.DataFrame({'track_id': track_id_list,'time_series' : audio_timeseries_list, 
                               'error_track' : error_list,'folder' : folder_list,'filepath' : filepath_list})

In [79]:
#alternative Getting summary statistic feature dictionaries for all numpy arrays stored in feature_dictionaries*** (*Do not run twice*
for feat_dict in feature_dict_0_50:
    track_id = feature_dict_0_50[i].split('_')[2].split('.')[0]
    
    load_feat_from = open('/Volumes/Extreme SSD/CAPSTONE_DATA/saved/features_dictionaries/{}'.format(feat_dict), "rb")
    a_feat_dict = pickle.load(load_feat_from)
    
    get_statistics_dict(a_feat_dict, reduce_limit = 50)
    
    save_ss_feat_dict_to = open('/Volumes/Extreme SSD/CAPSTONE_DATA/saved/ss_feature_dictionaries/dict_ss_feat_{}.pkl'.format(track_id), "wb")
    pickle.dump(dictionary, save_ss_feat_dict_to)
    save_ss_feat_dict_to.close()   
    

'dict_feat_000139.pkl'

In [59]:
df_time_series

,track_id,time_series,error_track,folder,filepath
0,000002,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,000,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
1,000003,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,000,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
2,000005,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,000,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
3,000010,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,000,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
4,000134,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,000,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
...,...,...,...,...,...
410,001965,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,001,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
411,001966,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,001,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
412,001967,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,001,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
413,001995,"([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",None,001,/Volumes/Extreme SSD/CAPSTONE_DATA/fma_medium/...
